In [70]:
# read data from mongodb
from pymongo import MongoClient
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

In [71]:
# connect to mongodb in localhost and access database and collections
def _connect_mongo():
    """ A util for making a connection to mongo """
    host = "localhost"
    port = 27017
    username = ""
    password = ""
    db = "5gopt"

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]


In [72]:
# get specific colelction, and remove the "_id" collumn
def collection_read_mongo(collection, query={}, no_id = True):
    db = _connect_mongo()
    cursor = db[collection].find(query)
    df = pd.DataFrame(list(cursor))

    if no_id:
        try:
            del df["_id"]
        except:
            pass
    return df

In [85]:
citiesName = ["Amsterdam", "Athens", "Barcelona", "Berlin", "Helsinki", "Paris", "Stockholm", 
              "London", "Manchester", "Roma", "Prague", "Edinburgh", "Vienna", "Lisbon", "Budapest",
             "Madrid", "Warsaw", "Brussels", "Dublin", "Munich", "Riga", "hamburg",
             "Vilnius", "Tallinn", "Reykjavik", "Frankfurt", "Zurich", 
              "liverpool", "Minsk", "Oslo", "Kiev", "Bucharest", "Sofia", "Cologne", "Kharkiv"]

In [86]:
len(citiesName)

35

In [88]:
"Roma" in citiesName

True

In [89]:
# convert cities name into all-lowercase name, and then check whether there are any duplicating names
citiesName = [i.lower() for i in citiesName]
duplicate = [i for i in citiesName if citiesName.count(i) > 1]
if len(duplicate) > 1:
    print("the list of cities name doesn't contain all cities, CHECK, CHECK")

citiesName = set(citiesName)

In [90]:
citiesName

{'amsterdam',
 'athens',
 'barcelona',
 'berlin',
 'brussels',
 'bucharest',
 'budapest',
 'cologne',
 'dublin',
 'edinburgh',
 'frankfurt',
 'hamburg',
 'helsinki',
 'kharkiv',
 'kiev',
 'lisbon',
 'liverpool',
 'london',
 'madrid',
 'manchester',
 'minsk',
 'munich',
 'oslo',
 'paris',
 'prague',
 'reykjavik',
 'riga',
 'roma',
 'sofia',
 'stockholm',
 'tallinn',
 'vienna',
 'vilnius',
 'warsaw',
 'zurich'}

In [77]:
# iterately access each mongodb collection and compare their data size, then rank them in order
client = MongoClient()

In [78]:
db = client.test

In [80]:
def changeCollectionName():
    '''
    this function only works for changing mongo collection name
    '''
    collNames = db.collection_names()
    for i in collNames:
        subNameList = i.split("_")
        if len(subNameList) > 1:
            if subNameList[1] in ["tripadvisor", "Tripadvisor"]:
#                 print("True")
                newName = subNameList[0].lower() + "Tripadvisor"
            if subNameList[1] in ["Booking", "booking"]:
                newName = subNameList[0].lower() + "Booking"
            else:#                 using original nameri
                newName = "_".join(subNameList)
            db[i].rename(newName)
        else:
            pass
    return db.collection_names()

In [81]:
print(db.collection_names())

['hamburgBooking', 'sofiaBooking', 'manchesterTripadvisor', 'frankfurtBooking', 'bucharestBooking', 'romeBooking', 'osloBooking', 'helsinkiBooking', 'london2Tripadvisor', 'paris2Tripadvisor', 'madridBooking', 'rigaBooking', 'parisTripadvisor', 'brusselsBooking', 'copenhagenBooking', 'athensBooking', 'munichBooking', 'stockholmBooking', 'barcelonaTripadvisor', 'copenhagenTripadvisor', 'berlinBooking', 'dublinTripadvisor', 'dublinBooking', 'berlinTripadvisor', 'warsawBooking', 'kharkivBooking', 'londonTripadvisor', 'budapestBooking', 'olsoTripadvisor', 'kievBooking', 'amsterdamTripadvisor', 'reykjavikTripadvisor', 'amsterdam3Tripadvisor', 'liverpoolBooking', 'athensTripadvisor', 'lisbonBooking', 'viennaBooking', 'reykjavikBooking', 'amsterdam2Tripadvisor', 'helsinkiTripadvisor', 'frankfurtTripadvisor', 'minskBooking', 'stockholmTripadvisor', 'pragueBooking', 'tallinnBooking', 'munichTripadvisor', 'brusselTripadvisor', 'parisBooking', 'edinburghBooking', 'liverpoolTripadvisor', 'edinburgh

In [124]:
tripadvisorSuffix = "Tripadvisor"
bookingSuffix = "Booking"
dataSizeList = list()
ByteToMB = 1024*1024
for i in citiesName:
    if i == "":
        pass
    else:
#         print(citiesName)
#         print(i)
        collTripadvisor = i+tripadvisorSuffix
        collBooking = i+bookingSuffix
    #     get the collection size and then convet to MB from bytes
    #     print(collTripadvisor)
#     since for booking or tripadvisor, some cities data doesn't be collected.
        try:
            size_in_trip = db.command("collstats", collTripadvisor)["size"] / ByteToMB
            countNum_in_trip = db.command("collstats", collTripadvisor)["count"]
        except:
            size_in_trip = None
            countNum_in_trip = None
#             pass
        try:
            size_in_booking = db.command("collstats", collBooking)["size"] / ByteToMB
            countNum_in_booking = db.command("collstats", collBooking)["count"]
        except:
            size_in_booking = None
            countNum_in_booking = None
#             pass
        dataSizeList.append([i, size_in_trip, countNum_in_trip, size_in_booking, countNum_in_booking])

dataSizeDF = pd.DataFrame(dataSizeList, columns=["CityName", "Tripadvisor(MB)","CountInTrip", "Booking(MB)", "CountInBooking"])
    
    

In [125]:
dataSizeDF

,CityName,Tripadvisor(MB),CountInTrip,Booking(MB),CountInBooking
0,vienna,NaN,NaN,55.802721,83411.0
1,athens,86.814761,79790.0,59.329007,91615.0
2,prague,NaN,NaN,55.812217,84239.0
3,helsinki,24.370316,23410.0,15.736367,23572.0
4,reykjavik,9.837871,7970.0,33.064641,48948.0
5,minsk,NaN,NaN,2.666970,4059.0
6,oslo,NaN,NaN,23.203299,36134.0
7,budapest,NaN,NaN,58.835356,88770.0
8,bucharest,NaN,NaN,15.988400,23710.0
9,barcelona,161.262398,143125.0,114.565245,172146.0


In [127]:
# sort dataframe only according to column "Booking.com"
top10Trpadvisor = dataSizeDF.sort_values(by="Tripadvisor(MB)", ascending=False)[:10][["CityName", "Tripadvisor(MB)", "CountInTrip"]]
top10Trpadvisor

,CityName,Tripadvisor(MB),CountInTrip
25,paris,504.663424,505681.0
21,amsterdam,224.306333,193163.0
14,dublin,174.130999,159551.0
9,barcelona,161.262398,143125.0
20,london,106.319992,94869.0
1,athens,86.814761,79790.0
3,helsinki,24.370316,23410.0
12,munich,20.163715,17912.0
24,frankfurt,19.770403,17863.0
4,reykjavik,9.837871,7970.0


In [129]:
# sort dataframe only according to column "Booking.com"
top10Booking = dataSizeDF.sort_values(by="Booking(MB)", ascending=False)[:10][["CityName", "Booking(MB)", "CountInBooking"]]
top10Booking

,CityName,Booking(MB),CountInBooking
22,edinburgh,128.539648,192517.0
9,barcelona,114.565245,172146.0
14,dublin,111.184878,173875.0
10,berlin,102.383070,151977.0
13,manchester,102.170732,152434.0
25,paris,86.797755,132728.0
17,liverpool,86.559738,134681.0
23,madrid,70.367912,108125.0
28,lisbon,61.561478,89450.0
1,athens,59.329007,91615.0


In [ ]:
#do statistics about review count. 

# remove review records which contain absent value

# print sample json data in booking json raw data

# sample json data in tripadvisor

In [32]:
print(db.command("collstats", "amsterdamTripadvisor")["size"])

235202237


34

In [ ]:
spark = SparkSession.builder.appName("statistics for hotels reviews in Booking").config(
    "spark.some.config.option", "booking").getOrCreate()